In [ ]:
!git clone https://github.com/JaidedAI/EasyOCR.git

Cloning into 'EasyOCR'...
remote: Enumerating objects: 2736, done.
remote: Total 2736 (delta 0), reused 0 (delta 0), pack-reused 2736
Receiving objects: 100% (2736/2736), 157.83 MiB | 25.84 MiB/s, done.
Resolving deltas: 100% (1664/1664), done.
Updating files: 100% (313/313), done.


In [ ]:
!pip install -r ./EasyOCR/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 24.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-man

In [ ]:
!mkdir "EasyOCR/user_network"
!mkdir "EasyOCR/model"
!mkdir "EasyOCR/newExamples"
!mkdir "EasyOCR/preprocessedData"

In [ ]:
%cd EasyOCR

/content/EasyOCR


In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from easyocr.easyocr import *
from PIL import Image

# GPU 설정
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'


def get_files(path):
    file_list = []

    files = [f for f in os.listdir(path) if not f.startswith('.')]  # skip hidden file
    files.sort()
    abspath = os.path.abspath(path)
    for file in files:
        file_path = os.path.join(abspath, file)
        file_list.append(file_path)

    return file_list, len(file_list)


if __name__ == '__main__':

    # # Using default model
    # reader = Reader(['ko'], gpu=True)

    # Using custom model
    #'/content/drive/MyDrive/Colab Notebooks/workspace/pre_trained_model/'
    reader = Reader(['ko'], gpu=True,
                    model_storage_directory='/content/drive/MyDrive/Colab Notebooks/workspace/pre_trained_model/',
                    user_network_directory='./user_network',
                    recog_network='custom')

    files, count = get_files('./output')

    for idx, file in enumerate(files):
        filename = os.path.basename(file)
        Image.MAX_IMAGE_PIXELS = None
        result = reader.readtext(file)


        # ./easyocr/utils.py 733 lines
        # result[0]: bbox
        # result[1]: string
        # result[2]: confidence
        for (bbox, string, confidence) in result:
            print("filename: '%s', confidence: %.4f, string: '%s'" % (filename, confidence, string))
            # print('bbox: ', bbox)

filename: '1.jpg', confidence: 0.2722, string: '여쭈'
filename: '10.jpg', confidence: 0.1319, string: '잇'
filename: '10.jpg', confidence: 0.9156, string: '1'
filename: '100.jpg', confidence: 0.7462, string: '같이'
filename: '101.jpg', confidence: 0.5100, string: '용히지'
filename: '102.jpg', confidence: 0.4853, string: '마쎄요'
filename: '103.jpg', confidence: 0.9287, string: '제된'
filename: '104.jpg', confidence: 0.9911, string: '악은'
filename: '105.jpg', confidence: 0.7996, string: '법'
filename: '106.jpg', confidence: 0.1903, string: '랑메'
filename: '107.jpg', confidence: 0.5043, string: '기재된'
filename: '108.jpg', confidence: 0.9954, string: '따'
filename: '109.jpg', confidence: 0.3217, string: '처빵기간'
filename: '11.jpg', confidence: 0.0403, string: '젠년윽'
filename: '110.jpg', confidence: 0.3969, string: '브요하'
filename: '112.jpg', confidence: 0.5107, string: '하세요'
filename: '113.jpg', confidence: 0.9991, string: '내'
filename: '114.jpg', confidence: 0.0282, string: '아껴워'
filename: '115.jpg', confiden

# 이미지 전처리

In [ ]:
from imutils.object_detection import non_max_suppression
from imutils.perspective import four_point_transform
from imutils.contours import sort_contours
import matplotlib.pyplot as plt
import imutils
import numpy as np
import requests
import cv2
import os
from scipy.ndimage import label

In [ ]:
def find_contours(img):
  blurred = cv2.GaussianBlur(img, (5, 5,), 0)
  ret, thr1 = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)
  # edged = cv2.Canny(blurred, 74, 200)
  # edged = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

  # plt.subplot(121),plt.imshow(img)
  # plt.subplot(122),plt.imshow(thr1)
  # plt.title("find_contours")
  # plt.show()
  # return gray
  return thr1

In [ ]:
def sharpening_img(img):
  sharpening_mask1 = np.array([[-2, -2, -2], [-2, 18, -2], [-2, -2, -2]])
  sharpening_out1 = cv2.filter2D(img, -1, sharpening_mask1)

  # plt.subplot(121),plt.imshow(img)
  # plt.subplot(122),plt.imshow(sharpening_out1)
  # plt.title("sharpening")
  # plt.show()
  return sharpening_out1

In [ ]:
def find_outlines(edged, img):
  cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

  roi_list = []
  roi_title_list = []

  margin = 20
  receipt_grouping = img.copy()

  for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w // float(h)

    if ar > 3.0 and ar < 6.5 and (W/2) < x:
      color = (0, 255, 0)
      roi = img[y - margin:y + h + margin, x - margin:x + w + margin]
      roi_list.append(roi)
      roi_title_list.append("Roi_{}".format(len(roi_list)))
    else:
      color = (0, 0, 255)

    cv2.rectangle(receipt_grouping, (x - margin, y - margin), (x + w + margin, y + h + margin), color, 2)
    cv2.putText(receipt_grouping, "".join(str(ar)), (x, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.65, color, 2)

  plt_imshow(["Grouping Image"], [receipt_grouping], figsize=(16, 10))

In [ ]:
def denoisiong_img(img):
  dst = cv2.fastNlMeansDenoising(img, None, 10, 7, 21)

  # plt.subplot(121),plt.imshow(img)
  # plt.subplot(122),plt.imshow(dst)
  # plt.title("denoisiong")
  # plt.show()
  return dst

In [ ]:
def plt_imshow(title='image', img=None, figsize=(8 ,5)):
  # 그림(figure) 객체를 생성(그림의 크기, 해상도, 배경색 등의 속성을 직접 지정하고 싶을 때 필수 생성-객체 이용해야 함)
  #figsize=(8,5) : 가로 8인치 세로 5인치 그림 생성
    plt.figure(figsize=figsize)

#img : cv2.imread 한 이미지

  # 이미지가 여러개 일때
    if type(img) == list:
      #title이 여러 개면 titles는 곧 title, title이 한 개면 그림마다 해당 title을 가져서 titles = [title, title,...] 중복 리스트
        if type(title) == list:
            titles = title
        else:
            titles = []

            for i in range(len(img)):
                titles.append(title)

      #cv2는 이미지를 BGR로 불러오고, plt는 RGB 이미지가 필요함
      #리스트 내 모든 이미지에 대해
        for i in range(len(img)):
          #이미지가 grayscale이면 RGB이미지로 바꾸기
            if len(img[i].shape) <= 2:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_GRAY2RGB)
          #이미지가 BGR이라면 RGB로 바꾸기
            else:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB)
          #plot 여러개 그리기
          #subplot(행, 열, 그릴 위치) , imshow -> plot
          #아마 plot 쓴 이유는 이미지 여러 개 그리기 위해서인 듯
            plt.subplot(1, len(img), i + 1), plt.imshow(rgbImg)
            plt.title(titles[i])
          #눈금 설정
            plt.xticks([]), plt.yticks([])

        plt.show()
  #이미지 하나일 때
    else:
        if len(img.shape) < 3:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        else:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        plt.imshow(rgbImg)
        plt.title(title)
        plt.xticks([]), plt.yticks([])
        plt.show()

def decode_predictions(scores, geometry):
  #scores의 크기를 받고 bounding box 사각형을 추출한뒤 confidencs scores에 대응해본다
  #scores의 크기
  (numRows, numCols) = scores.shape[2:4]
  rects = []
  confidences = []

  for y in range(0, numRows):
    scoresData = scores[0, 0, y]
    xData0 = geometry[0, 0, y]
    xData1 = geometry[0, 1, y]
    xData2 = geometry[0, 2, y]
    xData3 = geometry[0, 3, y]
    anglesData = geometry[0, 4, y]

    for x in range(0, numCols):
      #기준 확률보다 작을 경우 박스 버리기
      if scoresData[x] < min_confidence:
        continue


      (offsetX, offsetY) = (x * 4.0, y * 4.0)
     # prediciton에 대한 회전각을 구하고 sin,cosine을 계산한다
     # 글씨가 회전되어 있을때를 대비
      angle = anglesData[x]
      cos = np.cos(angle)
      sin = np.sin(angle)

      h = xData0[x] + xData2[x]
      w = xData1[x] + xData3[x]

      endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
      endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
      startX = int(endX - w)
      startY = int(endY - h)

      rects.append((startX, startY, endX, endY))
      confidences.append(scoresData[x])

  return (rects, confidences)

In [ ]:
width = 640
height = 640
min_confidence = 0.5
padding = 0.0

In [ ]:
def load_file(folder, file):
  path = os.path.join(folder, file)
  org_image = cv2.imread(path)

  # plt_imshow("Original", org_image)

  return org_image

In [ ]:
folder = "/content/EasyOCR/newExamples"
folder = "/content/drive/MyDrive/Colab Notebooks/workspace/pre_examples"

In [ ]:
files = os.listdir(folder)

for file in files:
  print(file)
  #read_file로 이미지 생성해서 array로 받아오기 / url or file path
  img = load_file(folder, file)
  org_image = resize_img(img)
  org_image = find_contours(org_image)
  # # find_outlinaes(org_image, img)
  # org_image = denoisiong_img(org_image)
  org_image = sharpening_img(org_image)
  cv2.imwrite(f"./preprocessedData/{file}", org_image)
  print(org_image.shape)

KakaoTalk_20240413_184140770.jpg
(1578, 2102)
(6312, 8408)
KakaoTalk_20240421_143645255.jpg
(3029, 2939)
(12116, 11756)
KakaoTalk_20240421_143618939_03.jpg
(720, 1280)
(2880, 5120)
KakaoTalk_20240501_200012369.jpg
(3024, 4032)
(12096, 16128)
KakaoTalk_20240430_145159867.jpg
(1490, 2220)
(5960, 8880)


In [ ]:
def resize_img(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  print(gray.shape)
  resized = cv2.resize(gray, (gray.shape[1]*4,gray.shape[0]*4))
  kernel1 = np.ones((5,5), np.uint8)
  kernel2 = np.ones((3,3), np.uint8)

  dilated = cv2.dilate(resized, kernel1, iterations=1)
  dilated = cv2.erode(dilated, kernel2, iterations=1)
  # print(dilated.shape)
  morph1 = cv2.morphologyEx(dilated, cv2.MORPH_OPEN, kernel2)
  morph2 = cv2.morphologyEx(morph1, cv2.MORPH_CLOSE, kernel1)
  # plt.subplot(221),plt.imshow(img)
  # plt.subplot(222),plt.imshow(dilated)
  # plt.subplot(223),plt.imshow(morph1)
  # plt.subplot(224),plt.imshow(morph2)

  # plt.title("resizing")
  # plt.show()
  return morph1

In [ ]:
import cv2
import numpy as np
from imutils.object_detection import non_max_suppression
import argparse
import time
import cv2
import shutil
import os

# 이미지 불러오기
image = cv2.imread("/content/EasyOCR/preprocessedData/KakaoTalk_20240413_184140770.jpg")
orig = image.copy()
(H, W) = image.shape[:2]

# 이미지 resize
(newW, newH) = (1280, 1280)
rW = W / float(newW)
rH = H / float(newH)

image = cv2.resize(image, (newW, newH))
(H, W) = image.shape[:2]

#로딩 east 모델
layerNames = [
    "feature_fusion/Conv_7/Sigmoid",
    "feature_fusion/concat_3"]

print("[INFO] loading EAST text detector...")
net = cv2.dnn.readNet("/content/drive/MyDrive/Colab Notebooks/workspace/pre_trained_model/frozen_east_text_detection.pb")

# net 입력 위해 blob 객체로 변경
blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
                             (123.68, 116.78, 103.94), swapRB=True, crop=False)

# net 입력
net.setInput(blob)
# 학습 결과
(scores, geometry) = net.forward(layerNames)

#confidence scores
(numRows, numCols) = scores.shape[2:4]
rects = []
confidences = []

for y in range(0, numRows):
    # extract the scores (probabilities), followed by the geometrical
    # data used to derive potential bounding box coordinates that
    # surround text
    scoresData = scores[0, 0, y]
    xData0 = geometry[0, 0, y]
    xData1 = geometry[0, 1, y]
    xData2 = geometry[0, 2, y]
    xData3 = geometry[0, 3, y]
    anglesData = geometry[0, 4, y]

    for x in range(0, numCols):
        # if our score does not have sufficient probability, ignore it
        if scoresData[x] < 0.5:
            continue

        # compute the offset factor as our resulting feature maps will
        # be 4x smaller than the input image
        (offsetX, offsetY) = (x * 4.0, y * 4.0)

        # extract the rotation angle for the prediction and then
        # compute the sin and cosine
        angle = anglesData[x]
        cos = np.cos(angle)
        sin = np.sin(angle)

        # use the geometry volume to derive the width and height of
        # the bounding box
        h = xData0[x] + xData2[x]
        w = xData1[x] + xData3[x]

        # compute both the starting and ending (x, y)-coordinates for
        # the text prediction bounding box
        endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
        endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
        startX = int(endX - w)
        startY = int(endY - h)

        # add the bounding box coordinates and probability score to
        # our respective lists
        rects.append((startX, startY, endX, endY))
        confidences.append(scoresData[x])

# apply non-maxima suppression to suppress weak, overlapping bounding
# boxes
boxes = non_max_suppression(np.array(rects), probs=confidences)

count = 1
# change box width and height -> positive will add pixels and vice-versa
box_width_padding = 40
box_height_padding = 40

temp_image = orig.copy()

# delete output folder
try:
    shutil.rmtree('output')
except Exception as e:
    do = "nothing"
print("!!")
# create empty output folder
uncreated = 1
while (uncreated):
    try:
        os.mkdir('output')
        print("!")
        uncreated = 0
    except Exception as e:
        do = "nothing"

# define crop object
class Crop(object):
    def __init__(self, startX, startY, endX, endY):
        self.startX = startX
        self.startY = startY
        self.endX = endX
        self.endY = endY

    def __eq__(self, other):
        diff = abs(self.startY - other.startY)
        if (diff <= 10):
            return self.startX == other.startX
        else:
            False

    def __lt__(self, other):
        diff = abs(self.startY - other.startY)
        if (diff <= 10):
            return self.startX < other.startX
        else:
            return self.startY < other.startY

croppedList = []

# loop over the bounding boxes
for (startX, startY, endX, endY) in boxes:
    # scale the bounding box coordinates based on the respective
    # ratios
    startX = int(startX * rW) - box_width_padding
    startY = int(startY * rH) - box_height_padding
    endX = int(endX * rW) + box_width_padding
    endY = int(endY * rH) + box_height_padding

    # draw the bounding box on the image
    cv2.rectangle(orig, (startX, startY), (endX, endY), (0, 255, 0), 2)

    # append to croppedList to sort the images
    croppedList.append(Crop(startX, startY, endX, endY))

croppedList = sorted(croppedList)

for img in croppedList:
    roi = temp_image[img.startY:img.endY, img.startX:img.endX]
    cv2.imwrite("output/" + str(count) + ".jpg", roi)
    count = count + 1
    print(count)

# show the output image
cv2.imwrite("output/Text Detection.jpg", orig)
cv2.waitKey(0)

AttributeError: 'NoneType' object has no attribute 'copy'

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')